In [2]:
# prompt: import torch
import cv2
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np


In [3]:
#data transforms  ---> first tensors,then Image normalisation
#these values are used to normalized the image tensors
mean = 0.1307
std = 0.3081

#transformer for train and test data
transform_original = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

#transformer for new image (inference)
transform_inference = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
#load train n test data
train_dataset = datasets.MNIST(
    root='./data',
    train = True,
    transform = transform_original,
    download =True
)

test_dataset = datasets.MNIST(
    root='./data',
    train = True,
    transform = transform_original,
    download =True
)

100%|██████████| 9912422/9912422 [00:00<00:00, 143376387.16it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 101453679.92it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 47003074.66it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4406784.36it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
batch_size= 100
epochs= 10
lr=0.001

#creating training ad test minibatches
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle = True
)

print(f"No of images in the train dataset: {len(train_dataset)}")
print(f"No of images in the test dataset: {len(test_dataset)}")
print(f"No of batches in the train dataset: {len(train_loader)}")
print(f"No of batches in the test dataset: {len(test_loader)}")

No of images in the train dataset: 60000
No of images in the test dataset: 60000
No of batches in the train dataset: 600
No of batches in the test dataset: 600


In [5]:
#defining the networ architecture
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # first conv layer
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)

    #batch normalization----> normalizing output feature maps on 1st convolution layer
    self.batchnorm1 = nn.BatchNorm2d(8)

    #ReLU
    self.relu = nn.ReLU()

    #max poooinmg
    self.maxpoool1 = nn.MaxPool2d(kernel_size=2)

    #after the maxpool layer the size become 14 x 14

    #second conv layer
    self.cnn2= nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=1, padding=1)

    self.batchnorm2 = nn.BatchNorm2d(32)

    self.relu = nn.ReLU()
    self.maxpoool2 = nn.MaxPool2d(kernel_size=2)

    #afterthe 2 nd convolution + Max pooling the size become 7 x 7


    ##introducing fully connected layer ===> 7 x7 x 32

    self.fc1 = nn.Linear(
        in_features= 1568,
        out_features=600  #u can decide it

    )

    ##adding dropout layer
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear (in_features=600, out_features=10)



  def forward(self, input_image):
    # Conv Layer 01
    out = self.cnn1(input_image)
    out = self.batchnorm1(out)
    out = self.relu(out)
    out = self.maxpoool1(out)


    # Conv Layer 02
    out = self.cnn2(out)
    out = self.batchnorm2(out)
    out = self.relu(out)
    out = self.maxpoool2(out)

    # Flatterning the feature maps
    out = out.view(-1, 1568)

    # Fully connected layers
    out = self.fc1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.fc2(out)

    return out














In [6]:
# Initializing the objects

cnn = CNN()

CUDA = torch.cuda.is_available()

if CUDA:
  cnn = cnn.cuda()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr)

In [7]:
# Training Loop
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

# Epochs
for epoch in range(epochs):
  # Single epoch
  iteration = 0
  correct_predictions = 0
  iteration_loss = 0

  # Putting the model into train mode
  cnn.train()

  print(f"\niteration {epoch+1} is starting...\n")

  #Training
  for i, (inputs, labels) in enumerate(train_loader):
    # A single mini batch (100 images + labels)
    inputs = Variable(inputs)
    labels = Variable(labels)

    # deploy images and labels into gpu
    if torch.cuda.is_available():
      inputs = inputs.cuda()
      labels = labels.cuda()

    optimizer.zero_grad()

    output = cnn(inputs) # outputs for 100 images

    loss_val = loss(output, labels) # Calculate the loss for 100 images

    iteration_loss = iteration_loss + loss_val.item()

    # back propagation
    loss_val.backward()

    # weight update
    optimizer.step()

    # Getting the number with highest probability
    _, predicted_numbers = torch.max(output, 1)

    correct_predictions = correct_predictions + (predicted_numbers == labels.data).sum()

    iteration = iteration +1

    train_accuracy.append((correct_predictions / len(train_dataset) * 100))

  print(f"Train Accuracy for Epoch{epoch+1}: {(correct_predictions / len(train_dataset) * 100)}")

  #Testing
  test_loss = 0
  test_correct = 0
  iteration = 0

  #Put the model into evaluation mode (disable the backpropagation features)
  cnn.eval()

  for i, (inputs, labels) in enumerate(test_loader):
    # A single mini batch (100 images + labels)
    inputs = Variable(inputs)
    labels = Variable(labels)

    # deploy images and labels into gpu
    if torch.cuda.is_available():
      inputs = inputs.cuda()
      labels = labels.cuda()

    output = cnn(inputs) # outputs for 100 images

    loss_val = loss(output, labels) # Calculate the loss for 100 images

    test_loss = test_loss + loss_val.item()

    # Getting the number with highest probability
    _, predicted_numbers = torch.max(output, 1)

    test_correct = test_correct + (predicted_numbers == labels.data).sum()

    iteration = iteration +1

    test_accuracy.append((test_correct / len(test_dataset) * 100))

  print(f"Test Accuracy for Epoch {epoch+1}: {(test_correct / len(test_dataset) * 100)}")



iteration 1 is starting...

Train Accuracy for Epoch1: 60.95833206176758
Test Accuracy for Epoch 1: 85.98500061035156

iteration 2 is starting...

Train Accuracy for Epoch2: 83.72000122070312
Test Accuracy for Epoch 2: 90.21833801269531

iteration 3 is starting...

Train Accuracy for Epoch3: 88.16166687011719
Test Accuracy for Epoch 3: 92.15166473388672

iteration 4 is starting...

Train Accuracy for Epoch4: 90.3183364868164
Test Accuracy for Epoch 4: 93.17166900634766

iteration 5 is starting...

Train Accuracy for Epoch5: 91.53666687011719
Test Accuracy for Epoch 5: 93.92666625976562

iteration 6 is starting...

Train Accuracy for Epoch6: 92.32833099365234
Test Accuracy for Epoch 6: 94.48333740234375

iteration 7 is starting...

Train Accuracy for Epoch7: 93.08833312988281
Test Accuracy for Epoch 7: 94.92166900634766

iteration 8 is starting...

Train Accuracy for Epoch8: 93.58167266845703
Test Accuracy for Epoch 8: 95.23500061035156

iteration 9 is starting...

Train Accuracy for E

In [9]:
#Saving the model
torch.save(cnn.state_dict(), 'CNN-MNIST.pth')

In [10]:
cnn.state_dict()

OrderedDict([('cnn1.weight',
              tensor([[[[ 0.1494,  0.1822, -0.0501],
                        [-0.0235,  0.1878,  0.0862],
                        [ 0.2558, -0.1348, -0.1823]]],
              
              
                      [[[-0.3021,  0.0761, -0.3139],
                        [ 0.1877, -0.2212,  0.0026],
                        [ 0.2406,  0.0936, -0.1457]]],
              
              
                      [[[-0.0865,  0.3402, -0.0071],
                        [ 0.1489,  0.0267,  0.2147],
                        [-0.2242, -0.1442,  0.2941]]],
              
              
                      [[[ 0.3012,  0.1860,  0.1121],
                        [-0.1966, -0.2535,  0.1511],
                        [-0.2035,  0.0466,  0.1371]]],
              
              
                      [[[-0.1231, -0.2719,  0.2541],
                        [ 0.0071, -0.2421, -0.1276],
                        [ 0.2862, -0.0761, -0.1364]]],
              
              
                